# Library download

In [ ]:
!pip install -q \
    transformers \
    datasets \
    accelerate \
    peft \
    bitsandbytes \
    evaluate \
    bert-score



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="colab_gpu_limited_100.json")["train"]
dataset = dataset.map(lambda x: {"instruction": x["question"], "output": x["answer"]})


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
system_prompt = (
    "You are a compassionate and helpful mental health assistant.\n"
    "Your role is to provide emotional support and practical guidance to users struggling with anxiety, depression, or related issues.\n"
    "Your tone is empathetic, calm, non-judgmental, and professional.\n\n"
)

def format_prompt(example):
    prompt = f"{system_prompt}User: {example['instruction']}\nAssistant:"
    return {"text": prompt + " " + example["output"]}

dataset = dataset.map(format_prompt)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# Falcon 1 b model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],  # key layer in Falcon
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
# Use EOS token as padding token (safe for causal models like Falcon)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

# training the model

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# 🔒
os.environ["WANDB_DISABLED"] = "true"

# ✅ Ensure Falcon plays well with Trainer
model.config.use_cache = False  # Prevents shape mismatch in LoRA + caching

# 🧠 Training settings
training_args = TrainingArguments(
    output_dir="./falcon-mentalhealth-lora",     # where to save model checkpoints
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,                                   # use float16 for speed
    logging_steps=5,
    save_strategy="epoch",
    report_to="none"                             # New method to disable WandB/logging warnings
)

# 🧪 Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 🚀 Train!
trainer.train()



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quanti

Step,Training Loss
5,1.821400
10,1.623300
15,1.314800
20,1.341800
25,1.280800
30,1.134900
35,1.216200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

TrainOutput(global_step=36, training_loss=1.3885406123267279, metrics={'train_runtime': 95.502, 'train_samples_per_second': 3.141, 'train_steps_per_second': 0.377, 'total_flos': 1040944547758080.0, 'train_loss': 1.3885406123267279, 'epoch': 2.8})

In [ ]:
model.save_pretrained("./falcon-mentalhealth-lora")
tokenizer.save_pretrained("./falcon-mentalhealth-lora")



('./falcon-mentalhealth-lora/tokenizer_config.json',
 './falcon-mentalhealth-lora/special_tokens_map.json',
 './falcon-mentalhealth-lora/vocab.json',
 './falcon-mentalhealth-lora/merges.txt',
 './falcon-mentalhealth-lora/added_tokens.json',
 './falcon-mentalhealth-lora/tokenizer.json')

In [ ]:
from tqdm import tqdm

model.config.use_cache = True  # 🛠 Re-enable for generation after training
tokenizer.pad_token = tokenizer.eos_token  # Avoid pad token warning

# 🔥 Turn OFF gradient checkpointing after training to fix .generate()
model.gradient_checkpointing_disable()
model.config.use_cache = True


def generate_response(prompt):
    encoded = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = encoded.input_ids.cuda()
    attention_mask = encoded.attention_mask.cuda()

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id  # Prevents padding warning
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()


    # Add system prompt back
system_prompt = (
    "You are a compassionate and helpful mental health assistant.\n"
    "Your role is to provide emotional support and practical guidance to users struggling with anxiety, depression, or related issues.\n"
    "Your tone is empathetic, calm, non-judgmental, and professional.\n\n"
)

# Generate predictions
predictions = []
references = []

for row in tqdm(dataset):
    prompt = f"{system_prompt}User: {row['instruction']}\nAssistant:"
    response = generate_response(prompt)
    predictions.append(response)
    references.append(row["output"])


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
100%|██████████| 100/100 [17:46<00:00, 10.67s/it]


In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=581ee33fb1b10b47662b7514454aacbd399101938065891206e7faaefd3db10b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


# results evaluation

In [ ]:
import evaluate

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# BLEU expects references as list of lists
bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("🟦 BLEU Score:", round(bleu_result["bleu"] * 100, 2))

# ROUGE - we extract specific scores
rouge_result = rouge.compute(predictions=predictions, references=references)
print("🟥 ROUGE-1:", round(rouge_result["rouge1"] * 100, 2))
print("🟥 ROUGE-2:", round(rouge_result["rouge2"] * 100, 2))
print("🟥 ROUGE-L:", round(rouge_result["rougeL"] * 100, 2))

# BERTScore - F1
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")
bertscore_f1_avg = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])
print("🟩 BERTScore (F1):", round(bertscore_f1_avg * 100, 2))


🟦 BLEU Score: 4.08
🟥 ROUGE-1: 36.65
🟥 ROUGE-2: 9.51
🟥 ROUGE-L: 19.12


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🟩 BERTScore (F1): 86.55


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving no_gpu_limit_500.json to no_gpu_limit_500.json


In [ ]:
from datasets import load_dataset

# Load full dataset
new_dataset = load_dataset("json", data_files="no_gpu_limit_500.json")["train"]

# Rename keys to match previous format if needed
new_dataset = new_dataset.map(lambda x: {"instruction": x["question"], "output": x["answer"]})

# Take last 100 rows
benchmark_dataset = new_dataset.select(range(len(new_dataset) - 100, len(new_dataset)))


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# bench mark dataset last 100 rows

In [ ]:
# Reuse previous setup
from tqdm import tqdm

model.gradient_checkpointing_disable()
model.config.use_cache = True
tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt):
    encoded = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = encoded.input_ids.cuda()
    attention_mask = encoded.attention_mask.cuda()

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()

# Add system prompt
system_prompt = (
    "You are a compassionate and helpful mental health assistant.\n"
    "Your role is to provide emotional support and practical guidance to users struggling with anxiety, depression, or related issues.\n"
    "Your tone is empathetic, calm, non-judgmental, and professional.\n\n"
)

# Generate predictions
benchmark_predictions = []
benchmark_references = []

for row in tqdm(benchmark_dataset):
    prompt = f"{system_prompt}User: {row['instruction']}\nAssistant:"
    response = generate_response(prompt)
    benchmark_predictions.append(response)
    benchmark_references.append(row["output"])


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 100/100 [17:52<00:00, 10.73s/it]


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

bleu_result = bleu.compute(predictions=benchmark_predictions, references=[[ref] for ref in benchmark_references])
print("🟦 BLEU:", round(bleu_result["bleu"] * 100, 2))

rouge_result = rouge.compute(predictions=benchmark_predictions, references=benchmark_references)
print("🟥 ROUGE-1:", round(rouge_result["rouge1"] * 100, 2))
print("🟥 ROUGE-2:", round(rouge_result["rouge2"] * 100, 2))
print("🟥 ROUGE-L:", round(rouge_result["rougeL"] * 100, 2))

bertscore_result = bertscore.compute(predictions=benchmark_predictions, references=benchmark_references, lang="en")
bert_f1 = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])
print("🟩 BERTScore (F1):", round(bert_f1 * 100, 2))


🟦 BLEU: 4.21
🟥 ROUGE-1: 35.94
🟥 ROUGE-2: 9.5
🟥 ROUGE-L: 18.59


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🟩 BERTScore (F1): 86.51


In [ ]:
from tabulate import tabulate

# Format both evaluations
scores = [
    ["Metric", "Normal Dataset", "Benchmark Dataset"],
    ["🟦 BLEU", "4.08", "4.21"],
    ["🟥 ROUGE-1", "36.65", "35.94"],
    ["🟥 ROUGE-2", "9.51", "9.50"],
    ["🟥 ROUGE-L", "19.12", "18.59"],
    ["🟩 BERTScore (F1)", "86.55", "86.51"]
]

print(tabulate(scores, headers="firstrow", tablefmt="fancy_grid"))


╒═══════════════════╤══════════════════╤═════════════════════╕
│ Metric            │   Normal Dataset │   Benchmark Dataset │
╞═══════════════════╪══════════════════╪═════════════════════╡
│ 🟦 BLEU           │             4.08 │                4.21 │
├───────────────────┼──────────────────┼─────────────────────┤
│ 🟥 ROUGE-1        │            36.65 │               35.94 │
├───────────────────┼──────────────────┼─────────────────────┤
│ 🟥 ROUGE-2        │             9.51 │                9.5  │
├───────────────────┼──────────────────┼─────────────────────┤
│ 🟥 ROUGE-L        │            19.12 │               18.59 │
├───────────────────┼──────────────────┼─────────────────────┤
│ 🟩 BERTScore (F1) │            86.55 │               86.51 │
╘═══════════════════╧══════════════════╧═════════════════════╛


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r ./falcon-mentalhealth-lora /content/drive/MyDrive/falcon-mentalhealth-lora


In [ ]:
!cp -r ./falcon-mentalhealth-lora /content/drive/MyDrive/falcon-mentalhealth-lora-v1


In [ ]:
!zip -r /content/drive/MyDrive/falcon-mentalhealth-lora.zip ./falcon-mentalhealth-lora


  adding: falcon-mentalhealth-lora/ (stored 0%)
  adding: falcon-mentalhealth-lora/adapter_model.safetensors (deflated 8%)
  adding: falcon-mentalhealth-lora/special_tokens_map.json (deflated 75%)
  adding: falcon-mentalhealth-lora/tokenizer.json (deflated 82%)
  adding: falcon-mentalhealth-lora/runs/ (stored 0%)
  adding: falcon-mentalhealth-lora/runs/May01_14-01-45_79a94357fadb/ (stored 0%)
  adding: falcon-mentalhealth-lora/runs/May01_14-01-45_79a94357fadb/events.out.tfevents.1746108108.79a94357fadb.2143.0 (deflated 64%)
  adding: falcon-mentalhealth-lora/runs/May01_14-04-44_79a94357fadb/ (stored 0%)
  adding: falcon-mentalhealth-lora/runs/May01_14-04-44_79a94357fadb/events.out.tfevents.1746108285.79a94357fadb.2143.1 (deflated 64%)
  adding: falcon-mentalhealth-lora/checkpoint-13/ (stored 0%)
  adding: falcon-mentalhealth-lora/checkpoint-13/adapter_model.safetensors (deflated 8%)
  adding: falcon-mentalhealth-lora/checkpoint-13/rng_state.pth (deflated 25%)
  adding: falcon-mentalhea

In [ ]:
import pandas as pd

benchmark_df = pd.DataFrame({
    "instruction": [row["instruction"] for row in benchmark_dataset],
    "expected_response": benchmark_references,
    "falcon_response": benchmark_predictions
})

benchmark_df.to_csv("falcon_benchmark_predictions.csv", index=False)


In [ ]:
!cp falcon_benchmark_predictions.csv /content/drive/MyDrive/falcon_benchmark_predictions.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# 8-bit config
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# Load Falcon-1B with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-rw-1b",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# Load tokenizer + LoRA adapter from Drive
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/falcon-mentalhealth-lora", trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/falcon-mentalhealth-lora")
model.eval()


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(50304, 2048)
        (h): ModuleList(
          (0-23): 24 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (query_key_value): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
     

In [ ]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()

# Example prompt

system_prompt = (
    "You are a compassionate and helpful mental health assistant.\n"
    "Your role is to provide emotional support and practical guidance to users struggling with anxiety, depression, or related issues.\n"
    "Your tone is empathetic, calm, non-judgmental, and professional.\n\n"
)

user_input = "I've been feeling overwhelmed and I can't sleep properly lately."
prompt = f"{system_prompt}User: {user_input}\nAssistant:"
response = generate_response(prompt)
print("🧠 Falcon Response:", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🧠 Falcon Response: It's understandable that you're feeling stressed and anxious. It's important to remember that these feelings are normal and that they're not a reflection of your character or personality.

It's also important to remember that there are many different factors that contribute to your feelings of anxiety and stress, such as your current situation, your past experiences, and your current mental health.

It's essential to take time to reflect on your thoughts and feelings, and to identify the underlying causes of your anxiety and stress.

It's also essential to seek professional help if necessary.

It's essential to remember that your feelings of anxiety and stress are temporary and that they will pass with time.

In the meantime, it's essential to take


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00


# Imrpoving the score

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving no_gpu_limit_500.json to no_gpu_limit_500.json


In [ ]:
from datasets import load_dataset

# Load from Colab or Drive
dataset = load_dataset("json", data_files="no_gpu_limit_500.json")["train"]

dataset = dataset.map(lambda x: {"instruction": x["question"], "output": x["answer"]})


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
system_prompt = (
    "You are a compassionate and helpful mental health assistant.\n"
    "Your role is to provide emotional support and practical guidance to users struggling with anxiety, depression, or related issues.\n"
    "Your tone is empathetic, calm, non-judgmental, and professional.\n\n"
)

def format_prompt(example):
    prompt = f"{system_prompt}User: {example['instruction']}\nAssistant:"
    return {"text": prompt}

dataset = dataset.map(format_prompt)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

model_id = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config
)

model = prepare_model_for_kbit_t



raining(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config
)

model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./falcon-mentalhealth-lora-v2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=5,            # ✅ increased from 3
    learning_rate=1e-4,            # ✅ lower learning rate
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quanti

Step,Training Loss
10,2.035500
20,1.540000
30,1.155500
40,0.987900
50,0.982400
60,1.074900
70,0.914600
80,0.970800
90,1.026000
100,0.903200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

TrainOutput(global_step=155, training_loss=1.0843600180841262, metrics={'train_runtime': 822.5683, 'train_samples_per_second': 3.039, 'train_steps_per_second': 0.188, 'total_flos': 9041346929098752.0, 'train_loss': 1.0843600180841262, 'epoch': 4.864})

In [ ]:
model.save_pretrained("./falcon-mentalhealth-lora-v2")
tokenizer.save_pretrained("./falcon-mentalhealth-lora-v2")


('./falcon-mentalhealth-lora-v2/tokenizer_config.json',
 './falcon-mentalhealth-lora-v2/special_tokens_map.json',
 './falcon-mentalhealth-lora-v2/vocab.json',
 './falcon-mentalhealth-lora-v2/merges.txt',
 './falcon-mentalhealth-lora-v2/added_tokens.json',
 './falcon-mentalhealth-lora-v2/tokenizer.json')

In [ ]:
from tqdm import tqdm
import torch

model.gradient_checkpointing_disable()
model.eval()
tokenizer.pad_token = tokenizer.eos_token

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to("cuda")
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()

predictions = []
references = []

for row in tqdm(dataset):
    prompt = f"{system_prompt}User: {row['instruction']}\nAssistant:"
    response = generate_response(prompt)
    predictions.append(response)
    references.append(row["output"])


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
  0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 500/500 [1:25:52<00:00, 10.30s/it]


In [ ]:
!pip install evaluate rouge_score bert_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=430c5b9e9449ec8d9868f8dbbdae658cdfc6b3a5281a3917072a83a91f44fb3c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in references])
rouge_result = rouge.compute(predictions=predictions, references=references)
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

print("🟦 BLEU:", round(bleu_result["bleu"] * 100, 2))
print("🟥 ROUGE-1:", round(rouge_result["rouge1"] * 100, 2))
print("🟥 ROUGE-2:", round(rouge_result["rouge2"] * 100, 2))
print("🟥 ROUGE-L:", round(rouge_result["rougeL"] * 100, 2))
print("🟩 BERTScore (F1):", round(sum(bertscore_result["f1"]) / len(bertscore_result["f1"]) * 100, 2))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🟦 BLEU: 2.02
🟥 ROUGE-1: 27.13
🟥 ROUGE-2: 6.21
🟥 ROUGE-L: 14.66
🟩 BERTScore (F1): 84.85
